![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [96]:
# Start your code here!
library(readr)
library(dplyr)
library(forcats)

In [97]:
ds_jobs <- read_csv('customer_train.csv')
head(ds_jobs)
str(ds_jobs)

Rows: 19158 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): city, gender, relevant_experience, enrolled_university, education_...
dbl  (4): student_id, city_development_index, training_hours, job_change

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NA,NA,1,36,1
29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
11561,city_21,0.624,NA,No relevant experience,Full time course,Graduate,STEM,5,NA,NA,never,83,0
33241,city_115,0.789,NA,No relevant experience,NA,Graduate,Business Degree,<1,NA,Pvt Ltd,never,52,1
666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
21651,city_176,0.764,NA,Has relevant experience,Part time course,Graduate,STEM,11,NA,NA,1,24,1


spc_tbl_ [19,158 × 14] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ student_id            : num [1:19158] 8949 29725 11561 33241 666 ...
 $ city                  : chr [1:19158] "city_103" "city_40" "city_21" "city_115" ...
 $ city_development_index: num [1:19158] 0.92 0.776 0.624 0.789 0.767 0.764 0.92 0.762 0.92 0.92 ...
 $ gender                : chr [1:19158] "Male" "Male" NA NA ...
 $ relevant_experience   : chr [1:19158] "Has relevant experience" "No relevant experience" "No relevant experience" "No relevant experience" ...
 $ enrolled_university   : chr [1:19158] "no_enrollment" "no_enrollment" "Full time course" NA ...
 $ education_level       : chr [1:19158] "Graduate" "Graduate" "Graduate" "Graduate" ...
 $ major_discipline      : chr [1:19158] "STEM" "STEM" "STEM" "Business Degree" ...
 $ experience            : chr [1:19158] ">20" "15" "5" "<1" ...
 $ company_size          : chr [1:19158] NA "50-99" NA NA ...
 $ company_type          : chr [1:19158] NA "Pvt Ltd" NA "Pvt Ltd" 

In [98]:
ds_jobs_clean<- ds_jobs%>%
mutate(across(where(is.character),as.factor),
	  student_id=as.integer(student_id),
	  training_hours=as.integer(training_hours),
	  job_change=as.integer(job_change),
	  city_development_index= as.double(city_development_index))


str(ds_jobs_clean)

tibble [19,158 × 14] (S3: tbl_df/tbl/data.frame)
 $ student_id            : int [1:19158] 8949 29725 11561 33241 666 21651 28806 402 27107 699 ...
 $ city                  : Factor w/ 123 levels "city_1","city_10",..: 6 78 65 15 51 58 50 84 6 6 ...
 $ city_development_index: num [1:19158] 0.92 0.776 0.624 0.789 0.767 0.764 0.92 0.762 0.92 0.92 ...
 $ gender                : Factor w/ 3 levels "Female","Male",..: 2 2 NA NA 2 NA 2 2 2 NA ...
 $ relevant_experience   : Factor w/ 2 levels "Has relevant experience",..: 1 2 2 2 1 1 1 1 1 1 ...
 $ enrolled_university   : Factor w/ 3 levels "Full time course",..: 2 2 1 NA 2 3 2 2 2 2 ...
 $ education_level       : Factor w/ 5 levels "Graduate","High School",..: 1 1 1 1 3 1 2 1 1 1 ...
 $ major_discipline      : Factor w/ 6 levels "Arts","Business Degree",..: 6 6 6 2 6 6 NA 6 6 6 ...
 $ experience            : Factor w/ 22 levels "<1",">20","1",..: 2 9 18 1 2 5 18 7 20 11 ...
 $ company_size          : Factor w/ 8 levels "<10","10-49",..: NA 6 

In [99]:


ds_jobs_clean <- ds_jobs_clean %>%
mutate(company_size= fct_collapse(company_size,
								 'Micro'='<10',
								 'Small'=c('10-49','50-99'),
								 'Medium'=c('100-499','500-999'),
								 'Large'=c('1000-4999','10000+','5000-9999')))
ds_jobs_clean <- ds_jobs_clean %>%
mutate(experience=fct_collapse(experience,
							 '<5'=c('<1','2','3','4','1'),
							  '5-10'=c('5','6','7','8','9','10'),
							  '>10'=c('11','12','13','14','15','16','17','18','19','20','>20')))

str(ds_jobs_clean)
levels(ds_jobs_clean$company_size)
levels(ds_jobs_clean$experience)

tibble [19,158 × 14] (S3: tbl_df/tbl/data.frame)
 $ student_id            : int [1:19158] 8949 29725 11561 33241 666 21651 28806 402 27107 699 ...
 $ city                  : Factor w/ 123 levels "city_1","city_10",..: 6 78 65 15 51 58 50 84 6 6 ...
 $ city_development_index: num [1:19158] 0.92 0.776 0.624 0.789 0.767 0.764 0.92 0.762 0.92 0.92 ...
 $ gender                : Factor w/ 3 levels "Female","Male",..: 2 2 NA NA 2 NA 2 2 2 NA ...
 $ relevant_experience   : Factor w/ 2 levels "Has relevant experience",..: 1 2 2 2 1 1 1 1 1 1 ...
 $ enrolled_university   : Factor w/ 3 levels "Full time course",..: 2 2 1 NA 2 3 2 2 2 2 ...
 $ education_level       : Factor w/ 5 levels "Graduate","High School",..: 1 1 1 1 3 1 2 1 1 1 ...
 $ major_discipline      : Factor w/ 6 levels "Arts","Business Degree",..: 6 6 6 2 6 6 NA 6 6 6 ...
 $ experience            : Factor w/ 3 levels "<5",">10","5-10": 2 2 3 1 2 2 3 2 3 2 ...
 $ company_size          : Factor w/ 4 levels "Micro","Small",..: NA 2 NA 

[1] "Micro"  "Small"  "Medium" "Large"

[1] "<5"   ">10"  "5-10"

In [100]:
ds_jobs_clean<-ds_jobs_clean%>%
filter(experience=='>10'& company_size=='Large')

In [101]:
object.size(ds_jobs)
object.size(ds_jobs_clean)

2167296 bytes

134768 bytes